# Imoprt CVS Information Page

Purpose of this Project: Extract Vaccine appointment info from the CVS. CVS data team has provide the links of appointment, slot and schedule. For more information please refer to https://developer.cvshealth.com/covid19/cdc/bulk-publish

Key Point:
Json File
Ndjson File


In [191]:
import requests
import json
import pandas as pd
import ndjson  ## Location Schedule and Slot are the Ndjson format
from datetime import datetime

In [192]:
re=requests.get("https://www.cvs.com/immunizations/inventory/data/$bulk-publish")

In [193]:
# Check the status
re.status_code

200

In [194]:
re.json()

{'transactionTime': '2021-05-14T20:02:01.534Z',
 'request': 'https://www.cvs.com/immunizations/inventory/data/$bulk-publish',
 'output': [{'type': 'Location',
   'url': 'https://www.cvs.com/immunizations/inventory/data/location_file_05142021130201.ndjson'},
  {'type': 'Schedule',
   'url': 'https://www.cvs.com/immunizations/inventory/data/schedule_file_05142021130201.ndjson'},
  {'type': 'Slot',
   'url': 'https://www.cvs.com/immunizations/inventory/data/slot_file_05142021130201.ndjson'}],
 'error': []}

In [195]:
re.json()['output']

[{'type': 'Location',
  'url': 'https://www.cvs.com/immunizations/inventory/data/location_file_05142021130201.ndjson'},
 {'type': 'Schedule',
  'url': 'https://www.cvs.com/immunizations/inventory/data/schedule_file_05142021130201.ndjson'},
 {'type': 'Slot',
  'url': 'https://www.cvs.com/immunizations/inventory/data/slot_file_05142021130201.ndjson'}]

In [196]:
Type=[]
Url=[]
for i in re.json()['output']:
    Type.append(i['type'])
    Url.append(i['url'])

In [197]:
Type

['Location', 'Schedule', 'Slot']

In [198]:
Url

['https://www.cvs.com/immunizations/inventory/data/location_file_05142021130201.ndjson',
 'https://www.cvs.com/immunizations/inventory/data/schedule_file_05142021130201.ndjson',
 'https://www.cvs.com/immunizations/inventory/data/slot_file_05142021130201.ndjson']

In [199]:
df=pd.DataFrame(Url,columns=['Url'],index=Type)
#set Type as index

In [200]:
df.loc['Location']["Url"]  # becasue the type is Index

'https://www.cvs.com/immunizations/inventory/data/location_file_05142021130201.ndjson'

In [201]:
df.loc['Schedule']["Url"]

'https://www.cvs.com/immunizations/inventory/data/schedule_file_05142021130201.ndjson'

In [202]:
df.loc['Slot']['Url']

'https://www.cvs.com/immunizations/inventory/data/slot_file_05142021130201.ndjson'

In [203]:
requests.get(df.loc['Location']["Url"])

<Response [200]>

# Import

In [204]:
Location_re=requests.get(df.loc['Location']["Url"])
# there are ndjson file
#Location_re.json()     will return no value, so I used .text without .json()
Schedule_re=requests.get(df.loc['Schedule']["Url"])
Slot_re=requests.get(df.loc['Slot']["Url"])

In [286]:
#Location_re.text

## Convert String to Ndjson Format

### Location

In [206]:
ndjson.loads(Location_re.text)[0]
#The ndjson file are different from json file, the boundary is ",", but also be careful of the ","

{'resourceType': 'Location',
 'id': '2502',
 'identifier': [{'system': 'https://cdc.gov/vaccines/programs/vtrcks',
   'value': 'CV1002502'}],
 'name': 'CVS Pharmacy in CHESHIRE, CT',
 'telecom': [{'system': 'phone', 'value': '888-607-4287'},
  {'system': 'url',
   'value': 'https://www.cvs.com/immunizations/covid-19-vaccine?cid=oc_vacfnd_cvd'}],
 'address': {'line': ['581 HIGHLAND AVE.'],
  'city': 'CHESHIRE',
  'state': 'CT',
  'postalCode': '06410'}}

In [207]:
ndjson.loads(Location_re.text)[0].keys()

dict_keys(['resourceType', 'id', 'identifier', 'name', 'telecom', 'address'])

In [208]:
ndjson.loads(Location_re.text)[0]['telecom']
#type(ndjson.loads(Location_re.text)[0]['telecom'])  ## It is the list

[{'system': 'phone', 'value': '888-607-4287'},
 {'system': 'url',
  'value': 'https://www.cvs.com/immunizations/covid-19-vaccine?cid=oc_vacfnd_cvd'}]

In [209]:
ndjson.loads(Location_re.text)[0]['telecom'][0]
#type(ndjson.loads(Location_re.text)[0]['telecom'][0]) ## It is the dict

{'system': 'phone', 'value': '888-607-4287'}

In [210]:
ndjson.loads(Location_re.text)[0]['telecom'][0]['value']

'888-607-4287'

### Schedule

In [211]:
ndjson.loads(Schedule_re.text)[0]

{'resourceType': 'Schedule',
 'id': 'C250B470EE32ED97E0536CF0D90A07A9',
 'serviceType': [{'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/service-type',
     'code': '57',
     'display': 'Immunization'},
    {'system': 'http://fhir-registry.smarthealthit.org/StructureDefinition/vaccine-dose',
     'code': 'covid19-immunization',
     'display': 'COVID-19 Immunization Appointment'}]}],
 'actor': [{'reference': 'Location/1482'}]}

In [212]:
ndjson.loads(Schedule_re.text)[0].keys()

dict_keys(['resourceType', 'id', 'serviceType', 'actor'])

In [213]:
ndjson.loads(Schedule_re.text)[0]['actor'][0]['reference'].split('/')[1]
# In the reference: the location/9714, we only need the number after /

'1482'

In [214]:
ndjson.loads(Schedule_re.text)[0]['id']

'C250B470EE32ED97E0536CF0D90A07A9'

### Slot

In [215]:
ndjson.loads(Slot_re.text)[0]

{'resourceType': 'Slot',
 'id': 'C250B471143DED97E0536CF0D90A07A9',
 'schedule': {'reference': 'Schedule/C250B470F1F3ED97E0536CF0D90A07A9'},
 'status': 'busy',
 'start': '2021-05-17T09:00:00.000Z',
 'end': '2021-05-17T19:00:00.000Z',
 'extension': [{'url': 'http://fhir-registry.smarthealthit.org/StructureDefinition/booking-deep-link',
   'valueUrl': 'https://www.cvs.com/immunizations/covid-19-vaccine?cid=oc_vacfnd_cvd'},
  {'url': 'http://fhir-registry.smarthealthit.org/StructureDefinition/slot-capacity',
   'valueInteger': '0'}]}

In [216]:
ndjson.loads(Slot_re.text)[0].keys()

dict_keys(['resourceType', 'id', 'schedule', 'status', 'start', 'end', 'extension'])

In [217]:
ndjson.loads(Slot_re.text)[0]['schedule']['reference'].split('/')[1]

'C250B470F1F3ED97E0536CF0D90A07A9'

In [218]:
ndjson.loads(Slot_re.text)[0]['status']

'busy'

In [219]:
ndjson.loads(Slot_re.text)[0]['start']

'2021-05-17T09:00:00.000Z'

In [220]:
ndjson.loads(Slot_re.text)[0]['end']

'2021-05-17T19:00:00.000Z'

In [221]:
ndjson.loads(Slot_re.text)[0]['extension'][1]['valueInteger']

'0'

## Create For Loop to organize information of Locations

In [222]:
Locations_Info=[]
for Locations in ndjson.loads(Location_re.text):
    Detail_Info={'id':Locations['id'],
     'Line':Locations['address']['line'],
     'City':Locations['address']['city'],
     'State':Locations['address']['state'],
     'PostalCode':Locations['address']['postalCode'],
         }
    Locations_Info.append(Detail_Info)

In [287]:
#Locations_Info

In [224]:
Location_DF=pd.DataFrame(Locations_Info)  ## Becasue Locations_Info is the List and dic is inside of this List

In [225]:
Location_DF

,id,Line,City,State,PostalCode
0,2502,[581 HIGHLAND AVE.],CHESHIRE,CT,06410
1,1521,[9140 ROTHBURY DRIVE],GAITHERSBURG,MD,20886
2,4365,[709 W. COLLEGE ST.],PULASKI,TN,38478
3,1087,[20 FILORS LN.],STONY POINT,NY,10980
4,4984,[863 BALTIMORE PIKE],GLEN MILLS,PA,19342
...,...,...,...,...,...
9712,3634,[2160 LEE RD],CLEVELAND HEIGHTS,OH,44118
9713,2535,[49 CLAREMONT AVE.],MONTCLAIR,NJ,07042
9714,8875,[3644 EAST OLYMPIC BOULEVARD],LOS ANGELES,CA,90023
9715,675,[159 N. MACDADE BLVD.],GLENOLDEN,PA,19036


In [235]:
Location_DF['State'].value_counts()
#Location_DF['State'].value_counts().sort_values()

CA    1155
TX     841
FL     836
NY     558
PA     454
MA     400
OH     385
IL     373
NC     359
VA     347
GA     342
NJ     342
IN     330
MI     297
MD     225
SC     206
MO     195
AZ     195
CT     179
AL     178
TN     158
LA     133
MN     128
NV      94
KY      80
WI      80
OK      76
HI      69
RI      61
DC      59
MS      57
WV      57
KS      54
CO      51
NH      47
WA      45
IA      38
NE      29
UT      29
ME      24
NM      24
AR      23
PR      22
OR      22
DE      20
MT      14
VT       9
ND       6
SD       3
AK       3
ID       3
WY       2
Name: State, dtype: int64

In [ ]:
Location_DF.groupby(["State","City"]).count().reset_index()

## Create For Loop to organize information of Schedules

In [ ]:
Schedule_Info=[]
for Schedules in ndjson.loads(Schedule_re.text):
    Detail_Info={'id':Schedules['id'],
                 'actor':Schedules['actor'][0]['reference'].split('/')[1]
         }
    Schedule_Info.append(Detail_Info)

In [ ]:
 Schedule_Info

In [ ]:
Schedule_DF=pd.DataFrame(Schedule_Info)

In [ ]:
Schedule_DF['id'].value_counts()

In [ ]:
Schedule_DF['actor'].value_counts().max()
#Schedule_DF['actor'].value_counts().max()    # 1

## Create For Loop to organize information of Slots

In [ ]:
Slot_Info=[]
for Slots in ndjson.loads(Slot_re.text):
    Detail_Info={'id':Slots['schedule']['reference'].split('/')[1],
                 'Status':Slots['status'],
                 'Start':Slots['start'],
                 'End':Slots['end'],
                 'ValueInteger':Slots['extension'][1]['valueInteger']
         }
    Slot_Info.append(Detail_Info)

In [ ]:
 Slot_Info

In [ ]:
Slot_DF=pd.DataFrame(Slot_Info)

In [ ]:
Slot_DF

In [ ]:
Slot_DF['id'].value_counts()
#Slot_DF['id'].value_counts().max()    ## 1
#The id is the unique value

In [258]:
Slot_DF['Start']=[datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ') for x in Slot_DF['Start']]

In [259]:
Slot_DF['End']=[datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ') for x in Slot_DF['End']]

In [ ]:
Slot_DF[['Status','ValueInteger']].drop_duplicates()
#Slot_DF['Status'].value_counts()
#Slot_DF['ValueInteger'].value_counts()
#Based on the result, the busy means 0, free means 1

In [ ]:
Slot_DF[['Start','End']].drop_duplicates()

# Combine 3 DataFrames Together

The location files are the all locations of CVS
The Schedule tels service type and locations(all of them the service types are same)
The slot file tells the free or busy 

In [244]:
Location_DF

,id,Line,City,State,PostalCode
0,2502,[581 HIGHLAND AVE.],CHESHIRE,CT,06410
1,1521,[9140 ROTHBURY DRIVE],GAITHERSBURG,MD,20886
2,4365,[709 W. COLLEGE ST.],PULASKI,TN,38478
3,1087,[20 FILORS LN.],STONY POINT,NY,10980
4,4984,[863 BALTIMORE PIKE],GLEN MILLS,PA,19342
...,...,...,...,...,...
9712,3634,[2160 LEE RD],CLEVELAND HEIGHTS,OH,44118
9713,2535,[49 CLAREMONT AVE.],MONTCLAIR,NJ,07042
9714,8875,[3644 EAST OLYMPIC BOULEVARD],LOS ANGELES,CA,90023
9715,675,[159 N. MACDADE BLVD.],GLENOLDEN,PA,19036


In [245]:
Schedule_DF

,id,actor
0,C2406DAB5EA443A6E0536CF0D90AD641,9714
1,C2406DAB5EA543A6E0536CF0D90AD641,7655
2,C2406DAB5EA643A6E0536CF0D90AD641,2000
3,C2406DAB5EA743A6E0536CF0D90AD641,5383
4,C2406DAB5EA843A6E0536CF0D90AD641,4615
...,...,...
9559,C2406DAB838443A6E0536CF0D90AD641,3634
9560,C2406DAB838543A6E0536CF0D90AD641,2535
9561,C2406DAB838643A6E0536CF0D90AD641,8875
9562,C2406DAB838743A6E0536CF0D90AD641,675


In [246]:
Slot_DF

,id,Status,Start,End,ValueInteger
0,C2406DAB7E1F43A6E0536CF0D90AD641,free,2021-05-17T09:00:00.000Z,2021-05-17T19:00:00.000Z,1
1,C2406DAB76C843A6E0536CF0D90AD641,busy,2021-05-14T09:00:00.000Z,2021-05-14T19:00:00.000Z,0
2,C2406DAB76C843A6E0536CF0D90AD641,free,2021-05-17T09:00:00.000Z,2021-05-17T19:00:00.000Z,1
3,C2406DAB76C843A6E0536CF0D90AD641,free,2021-05-16T09:00:00.000Z,2021-05-16T19:00:00.000Z,1
4,C2406DAB76C843A6E0536CF0D90AD641,busy,2021-05-15T09:00:00.000Z,2021-05-15T19:00:00.000Z,0
...,...,...,...,...,...
38251,C2406DAB763C43A6E0536CF0D90AD641,free,2021-05-17T09:00:00.000Z,2021-05-17T19:00:00.000Z,1
38252,C2406DAB763C43A6E0536CF0D90AD641,busy,2021-05-16T09:00:00.000Z,2021-05-16T19:00:00.000Z,0
38253,C2406DAB763C43A6E0536CF0D90AD641,busy,2021-05-15T09:00:00.000Z,2021-05-15T19:00:00.000Z,0
38254,C2406DAB834843A6E0536CF0D90AD641,busy,2021-05-14T09:00:00.000Z,2021-05-14T19:00:00.000Z,0


In [249]:
Schedule_DF['id'].isin(Slot_DF['id']).value_counts()
#True    9564
#Name: id, dtype: int64

True    9564
Name: id, dtype: int64

In [247]:
Location_DF.merge(Schedule_DF,how='left',left_on='id',right_on='actor')

,id_x,Line,City,State,PostalCode,id_y,actor
0,2502,[581 HIGHLAND AVE.],CHESHIRE,CT,06410,C2406DAB5C9643A6E0536CF0D90AD641,2502
1,1521,[9140 ROTHBURY DRIVE],GAITHERSBURG,MD,20886,C2406DAB5C9743A6E0536CF0D90AD641,1521
2,4365,[709 W. COLLEGE ST.],PULASKI,TN,38478,C2406DAB5C9843A6E0536CF0D90AD641,4365
3,1087,[20 FILORS LN.],STONY POINT,NY,10980,C2406DAB5C9943A6E0536CF0D90AD641,1087
4,4984,[863 BALTIMORE PIKE],GLEN MILLS,PA,19342,C2406DAB5C9A43A6E0536CF0D90AD641,4984
...,...,...,...,...,...,...,...
9712,3634,[2160 LEE RD],CLEVELAND HEIGHTS,OH,44118,C2406DAB838443A6E0536CF0D90AD641,3634
9713,2535,[49 CLAREMONT AVE.],MONTCLAIR,NJ,07042,C2406DAB838543A6E0536CF0D90AD641,2535
9714,8875,[3644 EAST OLYMPIC BOULEVARD],LOS ANGELES,CA,90023,C2406DAB838643A6E0536CF0D90AD641,8875
9715,675,[159 N. MACDADE BLVD.],GLENOLDEN,PA,19036,C2406DAB838743A6E0536CF0D90AD641,675


In [248]:
Schedule_DF.merge(Slot_DF,how='inner',left_on='id',right_on='id')

,id,actor,Status,Start,End,ValueInteger
0,C2406DAB5EA443A6E0536CF0D90AD641,9714,busy,2021-05-14T09:00:00.000Z,2021-05-14T19:00:00.000Z,0
1,C2406DAB5EA443A6E0536CF0D90AD641,9714,busy,2021-05-15T09:00:00.000Z,2021-05-15T19:00:00.000Z,0
2,C2406DAB5EA443A6E0536CF0D90AD641,9714,busy,2021-05-16T09:00:00.000Z,2021-05-16T19:00:00.000Z,0
3,C2406DAB5EA443A6E0536CF0D90AD641,9714,busy,2021-05-17T09:00:00.000Z,2021-05-17T19:00:00.000Z,0
4,C2406DAB5EA543A6E0536CF0D90AD641,7655,busy,2021-05-14T09:00:00.000Z,2021-05-14T19:00:00.000Z,0
...,...,...,...,...,...,...
38251,C2406DAB838743A6E0536CF0D90AD641,675,busy,2021-05-17T09:00:00.000Z,2021-05-17T19:00:00.000Z,0
38252,C2406DAB838A43A6E0536CF0D90AD641,2004,busy,2021-05-14T09:00:00.000Z,2021-05-14T19:00:00.000Z,0
38253,C2406DAB838A43A6E0536CF0D90AD641,2004,busy,2021-05-15T09:00:00.000Z,2021-05-15T19:00:00.000Z,0
38254,C2406DAB838A43A6E0536CF0D90AD641,2004,busy,2021-05-16T09:00:00.000Z,2021-05-16T19:00:00.000Z,0


In [272]:
DF1=Location_DF.merge(Schedule_DF,how='left',left_on='id',right_on='actor')
DF=DF1.merge(Slot_DF,how='inner',left_on='id_y',right_on='id')

In [279]:
DF=DF.drop(columns=['id_x','id_y'])

# Analysis

In [282]:
DF

,Line,City,State,PostalCode,actor,id,Status,Start,End,ValueInteger
0,[581 HIGHLAND AVE.],CHESHIRE,CT,06410,2502,C2406DAB5C9643A6E0536CF0D90AD641,busy,2021-05-14 09:00:00,2021-05-14 19:00:00,0
1,[581 HIGHLAND AVE.],CHESHIRE,CT,06410,2502,C2406DAB5C9643A6E0536CF0D90AD641,busy,2021-05-15 09:00:00,2021-05-15 19:00:00,0
2,[581 HIGHLAND AVE.],CHESHIRE,CT,06410,2502,C2406DAB5C9643A6E0536CF0D90AD641,busy,2021-05-16 09:00:00,2021-05-16 19:00:00,0
3,[581 HIGHLAND AVE.],CHESHIRE,CT,06410,2502,C2406DAB5C9643A6E0536CF0D90AD641,busy,2021-05-17 09:00:00,2021-05-17 19:00:00,0
4,[9140 ROTHBURY DRIVE],GAITHERSBURG,MD,20886,1521,C2406DAB5C9743A6E0536CF0D90AD641,busy,2021-05-14 09:00:00,2021-05-14 19:00:00,0
...,...,...,...,...,...,...,...,...,...,...
38251,[159 N. MACDADE BLVD.],GLENOLDEN,PA,19036,675,C2406DAB838743A6E0536CF0D90AD641,busy,2021-05-17 09:00:00,2021-05-17 19:00:00,0
38252,[3117 LOCKHEED BLVD.],ALEXANDRIA,VA,22306,2004,C2406DAB838A43A6E0536CF0D90AD641,busy,2021-05-14 09:00:00,2021-05-14 19:00:00,0
38253,[3117 LOCKHEED BLVD.],ALEXANDRIA,VA,22306,2004,C2406DAB838A43A6E0536CF0D90AD641,busy,2021-05-15 09:00:00,2021-05-15 19:00:00,0
38254,[3117 LOCKHEED BLVD.],ALEXANDRIA,VA,22306,2004,C2406DAB838A43A6E0536CF0D90AD641,busy,2021-05-16 09:00:00,2021-05-16 19:00:00,0


In [285]:
pd.pivot_table(DF,values='id',index='State',columns='Status',aggfunc='count')

Status,busy,free
State,,
AK,3.0,9.0
AL,392.0,308.0
AR,70.0,22.0
AZ,507.0,253.0
CA,4080.0,488.0
CO,154.0,50.0
CT,286.0,414.0
DC,227.0,9.0
DE,50.0,30.0
